In [5]:
import pygame
import random
import time
from datetime import datetime

# 1. 초기화 -> 변수선언
pygame.init()

# 2. 게임화면 설정 -> 크기 고정
size = [400, 900]  # 화면 크기(좌측 상단 좌표(0,0))
screen = pygame.display.set_mode(size)
title = "Using PyGame"

pygame.display.set_caption(title)

# 3. 게임 화면 내부 설정 -> 변수
clock = pygame.time.Clock()
font = pygame.font.Font("C:/Windows/Fonts/consolab.ttf", 22)

class Object:  # 이미지 추가
    def __init__(self):
        self.x = 0
        self.y = 0
        self.distance = 0
    
    def add_img(self, address):
        if address[-3:] == 'png':
            self.image = pygame.image.load(address).convert_alpha()
        else:
            self.image = pygame.image.load(address)
    
    def change_size(self, width, heigth):
        self.image = pygame.transform.scale(self.image, (width, heigth))
        self.width, self.height = self.image.get_size()
    
    def show(self):
        screen.blit(self.image, (self.x, self.y))

# a.x - b.width <= b.x <= a.x + a.width
# a.y - b.heigth <= b.y <= a.y + a.height
def crash(a, b):  # 충돌 판정
    if (a.x - b.width <= b.x) and (b.x <= a.x + a.width):
        if (a.y - b.height <= b.y) and (b.y <= a.y + a.height):
            return True
        else:
            return False
    else:
        return False

player = Object()
player.add_img("C:/Users/Administrator/Pictures/pygame/player.png")
player.change_size(50, 80)
player.x = round(size[0] / 2) - round(player.width / 2)
player.y = size[1] - player.height - 50
player.distance = 10

left_move = False
right_move = False
space_move = False

red = (255, 0, 0)
yellow = (255, 255, 0)
white = (255, 255, 255)  # 배경색; 하양
black = (0, 0, 0)  # 배경색; 검정

rocket_list = []
enemy_list = []

k = 0 # 미사일 조절
game_over = 0

score = 0  # 미사일로 적 제거시 1증가
miss = 0  # 적이 화면 밖으로 나갈 시 1증가

# 시작 대기
wait_exit = 0
while wait_exit == 0:
    clock.tick(60)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            
        if event.type == pygame.KEYDOWN:  # 키 누름
            if event.key == pygame.K_SPACE: # space-bar
                wait_exit = 1
                space_move = True
                k = 0
    
    font1 = pygame.font.Font("C:/Windows/Fonts/consolab.ttf", 25)
    text_press = font1.render("press 'space'!", True, red)
    screen.blit(text_press, (round(size[0] / 2 - size[0] / 4), round(size[1] / 2)))
    pygame.display.flip()
        
# 4. 메인 이벤트
start_time = datetime.now()

system_exit = 0

while(system_exit == 0):
#     print(score)
#     print("miss : {}".format(miss))
    #  4 - 1) FPS(Frame Per Second) 설정
    clock.tick(60)
    
    #  4 - 2) 입력(키보드, 마우스) 감지
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            
        if event.type == pygame.KEYDOWN:  # 키 누름
            if event.key == pygame.K_LEFT:  # ←
                left_move = True
            if event.key == pygame.K_RIGHT: # →
                right_move = True
            if event.key == pygame.K_SPACE: # space-bar
                space_move = True
                k = 0
        
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                left_move = False
            if event.key == pygame.K_RIGHT:
                right_move = False
            if event.key == pygame.K_SPACE:
                space_move = False

    #  4 - 3) 변화(입력 / 시간에 따른 변화)
    current_time = datetime.now()
    delta_time = round((current_time - start_time).total_seconds())
    
    if left_move == True:
        player.x -= player.distance
        if player.x <= 0:
            player.x = 0
    elif right_move == True:
        player.x += player.distance
        if player.x >= size[0] - player.width:
            player.x = size[0] - player.width
    
    if space_move == True and k % 6 == 0:
        rocket = Object()
        rocket.add_img("C:/Users/Administrator/Pictures/pygame/rocket.png")
        rocket.change_size(10, 20)
        rocket.x = round(player.x + player.width / 2 - rocket.width / 2)
        rocket.y = player.y - rocket.height - 10
        rocket.distance = 17
        rocket_list.append(rocket)
    
    k += 1
    
    delete_list = []
    for i in range(len(rocket_list)):  # 화면 밖으로 나가는 로켓 파악
        r = rocket_list[i]
        r.y -= r.distance
        if r.y <= -r.height:
            delete_list.append(r)
    try:
        delete_list.reverse()
        for d in delete_list:
            del rocket_list[d]
    except:
        pass
    
    if random.random() > 0.98:
        enemy = Object()
        enemy.add_img("C:/Users/Administrator/Pictures/pygame/enemy.png")
        enemy.change_size(40, 40)
        enemy.x = random.randrange(0, size[0] - enemy.width - round(player.width / 2))
        enemy.y = 15
        enemy.distance = 3
        enemy_list.append(enemy)
        
    for i in range(len(enemy_list)):  # 화면 밖으로 나가는 적 파악
            e = enemy_list[i]
            e.y += e.distance
            if e.y >= size[1]:
                delete_list.append(i)
    try:
        delete_list.reverse()
        for d in delete_list:
            del enemy_list[d]
            miss += 1
    except:
        pass
    
    delete_rocket_list = []
    delete_enemy_list = []
    for i in range(len(rocket_list)):    # 적과 로켓 충돌 파악 - 어떤 객체끼리 충돌했는지
        for j in range(len(enemy_list)):
            r = rocket_list[i]
            e = enemy_list[j]
            if crash(r, e) == True:
                delete_rocket_list.append(i)
                delete_enemy_list.append(j)
    
    delete_rocket_list = list(set(delete_rocket_list))  # 중복 제거
    delete_enemy_list = list(set(delete_enemy_list))  # 중복 제거
    
    try:
        delete_rocket_list.reverse()
        delete_enemy_list.reverse()
        
        for dr in delete_rocket_list:
            del rocket_list[dr]
        for de in delete_enemy_list:
            del enemy_list[de]
            score += 1
    except:
        pass
    
    for i in range(len(enemy_list)):  # 적과 player가 충돌했는지
        e = enemy_list[i]
        if crash(e, player) == True:
            system_exit = 1
            game_over = 1
#             time.sleep(1)
        
    #  4 - 4) 전사작업(그리기)
    screen.fill(black)
    player.show()
    
    for r in rocket_list:
        r.show()
    
    for e in enemy_list:
        e.show()
    
    text_score = font.render("score : {}, miss : {}".format(score, miss), True, white)
    screen.blit(text_score, (10, 10))
    
    text_time = font.render("time : {}".format(delta_time), True, yellow)
    screen.blit(text_time, (260, 10))
    
    #  4 - 5) 업데이트
    pygame.display.flip()

# 5. 종료
while game_over == 1 or system_exit == 1:
    screen.fill(white)
    text_over = font.render("game over!", True, red)
    screen.blit(text_over, (round(size[0] / 2 - size[0] / 4), round(size[1] / 2)))
    pygame.display.flip()

    time.sleep(3)
    pygame.quit()

error: display Surface quit